Из-за низкой скорости обработки одного батча (в перещете на один текст присерно 10 сек.), код этого и последующих этапов делался в облаке на Tesla A10

In [1]:
import torch
import uuid
import os
import evaluate
from datasets import Dataset
from transformers import (
    T5ForConditionalGeneration,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments
)
from peft import LoraConfig, get_peft_model, TaskType # type: ignore
from tqdm import tqdm

Из-за того, что мы создаем датасет с помощью матчинга негативных текстов в какой-либо текст из позитивной или нейтральной группы, неизбежно появляются ошибки связанные с тем, что в оригинальном тексте и таргете, может в принципе говориться про разные продукты.

Чтобы хоть как-то передать информацию о желаемым результатом, я предлагаю включить в промт информацию о продукте и типу текста

In [2]:
train = Dataset.from_parquet('./__output__/matched_train.parquet')
test = Dataset.from_parquet('./__output__/matched_test.parquet')

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [3]:
train

Dataset({
    features: ['nmId', 'text', 'responder', 'type', 'product_name', 'product_category_2', 'product_category_1', 'product_color', 'product_description', 'product_brand', 'vector', 'toxicity', 'emotions', 'target_nmId', 'target_text', 'target_responder', 'target_type', 'target_product_name', 'target_product_category_2', 'target_product_category_1', 'target_product_color', 'target_product_description', 'target_product_brand', 'target_vector', 'target_toxicity', 'target_emotions', '__index_level_0__'],
    num_rows: 15147
})

In [4]:
test

Dataset({
    features: ['nmId', 'text', 'responder', 'type', 'product_name', 'product_category_2', 'product_category_1', 'product_color', 'product_description', 'product_brand', 'vector', 'toxicity', 'emotions', 'target_nmId', 'target_text', 'target_responder', 'target_type', 'target_product_name', 'target_product_category_2', 'target_product_category_1', 'target_product_color', 'target_product_description', 'target_product_brand', 'target_vector', 'target_toxicity', 'target_emotions', '__index_level_0__'],
    num_rows: 1683
})

Так как у нас задача Seq2Seq модель, самым логичным вариантом будет выбрать какую-либо Encoder-Decoder модель, из которых самая популярная это flan-t5

In [5]:
model = T5ForConditionalGeneration.from_pretrained(
    'google/flan-t5-small',
    device_map='cuda:0',
    torch_dtype=torch.bfloat16,
)
tokenizer = AutoTokenizer.from_pretrained(
    'google/flan-t5-small', 
    useFast=True
)

coll = DataCollatorForSeq2Seq(model=model, tokenizer=tokenizer)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [6]:
sum([p.numel() for p in model.parameters() if p.requires_grad])

76961152

In [7]:
peft_conf = LoraConfig(
    TaskType.SEQ_2_SEQ_LM,
    r=32,
    lora_alpha=32,
    target_modules=['q', 'v'],
    lora_dropout=0.1,
    bias='none',
    inference_mode=False
)

peft_model = get_peft_model(model, peft_conf)

In [8]:
sum([p.numel() for p in peft_model.parameters() if p.requires_grad])

1376256

In [9]:
max([len(tokenizer.tokenize(row['text'])) for row in train]) # type: ignore

Token indices sequence length is longer than the specified maximum sequence length for this model (589 > 512). Running this sequence through the model will result in indexing errors


931

In [10]:
max([len(tokenizer.tokenize(row['target_text'])) for row in train]) # type: ignore

817

In [11]:
def preprocess(e):
    out = tokenizer(
        f'''
responder: {e['target_responder']}
type: {e['target_type']}
product_name: {e['target_product_name']}
product_category_2: {e['target_product_category_2']}
product_color: {e['target_product_color']}
product_brand: {e['target_product_brand']}
product_description: {e['target_product_description']}
toxicity: {e['toxicity']}
emotions: {e['emotions']}
text: {e['text']}
        ''',
        truncation=True,
        max_length=1500,
        return_tensors='pt'
    )

    out['input_ids'] = out['input_ids'][0] # type: ignore
    out['attention_mask'] = out['attention_mask'][0] # type: ignore

    labels = tokenizer(
        text_target=e['target_text'], 
        max_length=1500,         
        truncation=True,
        return_tensors='pt'
    )
    out['labels'] = labels['input_ids'][0] # type: ignore
    
    return out

In [12]:
train = train.map(preprocess, remove_columns=train.column_names) # type: ignore
test = test.map(preprocess, remove_columns=test.column_names) # type: ignore

train

Map:   0%|          | 0/15147 [00:00<?, ? examples/s]

Map:   0%|          | 0/1683 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 15147
})

In [13]:
checkpoint = str(uuid.uuid4())

checkpoint

'8fd9c01d-3ad4-4dcf-b00c-ccb399091a74'

In [14]:
os.makedirs(f'./models/{checkpoint}')

In [15]:
args = Seq2SeqTrainingArguments(
    output_dir=f'./models/{checkpoint}/runs',
    eval_strategy='epoch',
    per_device_train_batch_size=5,
    per_device_eval_batch_size=5,
    num_train_epochs=5,
    learning_rate=0.001,
    warmup_ratio=0.1,
    weight_decay=0.01,
    save_strategy='no',
    remove_unused_columns=False,
    bf16=True,
    predict_with_generate=True,
    generation_max_length=300
)

trainer = Seq2SeqTrainer(
    model=peft_model, # type: ignore
    args=args,
    data_collator=coll,
    train_dataset=train, # type: ignore
    eval_dataset=test, # type: ignore
)

In [16]:
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,2.101300,1.937135
2,2.017700,1.856336
3,1.968600,1.819550
4,1.930100,1.794183
5,1.933900,1.783127


TrainOutput(global_step=15150, training_loss=2.0216667089682603, metrics={'train_runtime': 3705.7868, 'train_samples_per_second': 20.437, 'train_steps_per_second': 4.088, 'total_flos': 3.591972182035661e+16, 'train_loss': 2.0216667089682603, 'epoch': 5.0})

In [17]:
trainer.save_model(f'./models/{checkpoint}/model')

In [18]:
meteor = evaluate.load('evaluate-metric/meteor')
rouge = evaluate.load('evaluate-metric/rouge')
bleu = evaluate.load('evaluate-metric/bleu')

[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/ubuntu/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [19]:
targets = []
preds = []
for row in tqdm(test):
    text_input_ids = torch.LongTensor([row['input_ids']]).to('cuda:0')
    text_attention_mask = torch.LongTensor([row['attention_mask']]).to('cuda:0')

    model_out = model.generate(
        input_ids=text_input_ids, 
        attention_mask=text_attention_mask,
        top_k=1,
        do_sample=True
    )[0]

    preds.append(tokenizer.decode(model_out, skip_special_tokens=True))
    targets.append(tokenizer.decode(row['labels'], skip_special_tokens=True))

  0%|          | 0/1683 [00:00<?, ?it/s]/home/ubuntu/.venv/lib/python3.12/site-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 1683/1683 [10:28<00:00,  2.68it/s]


In [20]:
print(f'''
Test metrice:
Meteor: {meteor.compute(predictions=preds, references=targets)['meteor']}
Rouge: {rouge.compute(predictions=preds, references=targets)['rougeLsum']},
Bleu: {bleu.compute(predictions=preds, references=targets)['bleu']}
''')


Test metrice:
Meteor: 0.06076279690497971
Rouge: 0.0030566091432843905,
Bleu: 5.0306628085742025e-06

